In [67]:
from selenium import webdriver
import csv
import re
import pandas as pd
import numpy as np
from random import sample

In [50]:
sp500 = pd.read_csv('./sp_500.csv')

In [86]:
sp_tickers = sp500.Ticker.drop([505,506,507],axis=0)

In [102]:
tickers = sp_tickers.sample(100).tolist()

['FCX',
 'MAS',
 'HAS',
 'AMZN',
 'MU',
 'VAR',
 'PEG',
 'LEG',
 'JBHT',
 'CAT',
 'K',
 'HES',
 'NDAQ',
 'BLL',
 'ACN',
 'AAP',
 'VTR',
 'LW',
 'PG',
 'ADI',
 'ORCL',
 'NSC',
 'RF',
 'FITB',
 'PBCT',
 'BHF',
 'AMAT',
 'MAR',
 'INCY',
 'ORLY',
 'ROL',
 'FFIV',
 'JWN',
 'NEE',
 'QRVO',
 'PVH',
 'CVX',
 'HUM',
 'NBL',
 'BDX',
 'AWK',
 'SJM',
 'WM',
 'V',
 'ES',
 'PXD',
 'CTSH',
 'CHD',
 'AMP',
 'DRI',
 'MS',
 'AEE',
 'ABMD',
 'ETN',
 'L',
 'CTXS',
 'HRS',
 'HSY',
 'MDLZ',
 'UNH',
 'UDR',
 'NWSA',
 'CCI',
 'VRSK',
 'AEP',
 'GPS',
 'JNJ',
 'CB',
 'LOW',
 'CMCSA',
 'MMM',
 'HLT',
 'PPG',
 'PFG',
 'ADP',
 'AJG',
 'EW',
 'AZO',
 'SNA',
 'ALB',
 'CHRW',
 'F',
 'UPS',
 'UAL',
 'FDX',
 'DISH',
 'MTD',
 'FAST',
 'ROP',
 'LNT',
 'XOM',
 'WU',
 'FLR',
 'PRU',
 'COO',
 'ADS',
 'APC',
 'CPRI',
 'CSX',
 'ABT']

In [114]:
with open('tickers.txt', 'w') as file:
    for item in tickers:
        file.writelines(f'{item}\n')

In [115]:
with open('tickers.txt','r') as file:
    tickerz = file.readlines()

In [170]:
main_page = 'https://www.secform4.com'
search_bar_xpath = '//*[@id="cikform"]/div/input[1]'
search_button_xpath = '//*[@id="cikform"]/div/input[2]'

In [176]:
driver = webdriver.Chrome()

In [177]:
driver.get(main_page)

In [178]:
for ticker in tickers[86:]:
    
    driver.get(main_page)
    search_bar_select = driver.find_element_by_xpath(search_bar_xpath)
    search_bar_select.click()
    search_bar_select.send_keys(ticker)
    search_bar_select = driver.find_element_by_xpath(search_button_xpath)
    search_bar_select.click()
    
    if driver.current_url != f'https://www.secform4.com/insider/showhistory.php?cik={ticker}':
        data_selection = driver.find_element_by_xpath('//*[@id="SecForm4Page"]/table[2]/tbody/tr/td[1]/form[1]/select/option[23]')
        data_selection.click()
        data_selection = driver.find_element_by_xpath('//*[@id="SecForm4Page"]/table[2]/tbody/tr/td[1]/form[1]/input[1]')
        data_selection.click()
        pagecount = driver.find_element_by_xpath('//*[@id="SecForm4Page"]/div[4]').text
        pagecount = pagecount.split(',')
        if len(pagecount) <= 1:
            continue
        pagecount = int(pagecount[-2].strip())
        csv_file = open('insiders_{}.csv'.format(ticker),'w',encoding='utf-8',newline='',)
        writer = csv.writer(csv_file)
        index = 0

        while index <= pagecount:

            print('Scraping page number'+str(index))
            index += 1

            rows = driver.find_elements_by_xpath('//*[@id="filing_table"]//tr')
            rows = rows[1:]

            for item in rows:
                row_list = []
                row_dict = {}

                for i in range(1,10):


                    row_xpath = './/td[{}]'.format(str(i))
                    row = item.find_element_by_xpath(row_xpath).text
                    row_list.append(row)

                transaction_date = row_list[0].split()[0]
                transaction_type = row_list[0].split()[1]
                company_name = ' '.join(row_list[2].split())
                ticker = row_list[3]
                insider_name = ' '.join(row_list[4].split('\n')[-2:-len(row_list[4])-1:-1])

                insider_position = row_list[4].split('\n')[-1]
                shares_traded = int(row_list[5].replace(',',''))
                avg_price = float(row_list[6].replace('$','').replace(',',''))
                total_price = float(row_list[7].replace('$','').replace(',',''))
                shares_owned = int(row_list[8].split()[0].replace(',',''))
                ownership_type = row_list[8].split()[1].strip('()')

                row_dict['transaction_date'] = transaction_date
                row_dict['transaction_type'] = transaction_type
                row_dict['company_name'] = company_name
                row_dict['ticker'] = ticker
                row_dict['insider_name'] = insider_name
                row_dict['insider_position'] = insider_position
                row_dict['shares_traded'] = shares_traded
                row_dict['avg_price'] = avg_price
                row_dict['total_price'] = total_price
                row_dict['shares_owned'] = shares_owned
                row_dict['onwership_type'] = ownership_type

                writer.writerow(row_dict.values())


            try:

                next_button = driver.find_element_by_link_text('Next')
                next_button.click()

            except:

                break
                
    else:
        continue
    
    csv_file.close()




Scraping page number0
Scraping page number1
Scraping page number2
Scraping page number3
Scraping page number4
Scraping page number5
Scraping page number6
Scraping page number7
Scraping page number8
Scraping page number9
Scraping page number10
Scraping page number11
Scraping page number12
Scraping page number13
Scraping page number14
Scraping page number15
Scraping page number16
Scraping page number17
Scraping page number0
Scraping page number1
Scraping page number2
Scraping page number3
Scraping page number4
Scraping page number5
Scraping page number6
Scraping page number7
Scraping page number8
Scraping page number9
Scraping page number10
Scraping page number11
Scraping page number12
Scraping page number13
Scraping page number14
Scraping page number15
Scraping page number0
Scraping page number1
Scraping page number2
Scraping page number3
Scraping page number4
Scraping page number5
Scraping page number6
Scraping page number7
Scraping page number8
Scraping page number9
Scraping page numb

IndexError: list index out of range